# Modelling crystallisation in magmatic systems
**JAR625M 2025 - Week 10 - Practical 1**

Simon Matthews (simonm@hi.is)

---

In this practical you will apply thermodynamic models of magma and crystals to predict the chemical evolution of magmas during cooling and crystallisation.

The volcano Hekla is at the margin of Iceland's Eastern Volcanic Zone ([see it on the geological map](https://arcgisserver.isor.is/?lon=-19.65797&lat=63.98897&zoom=13&_ga=2.98804317.2103429531.1682513440-734776706.1680257934&_gl=1*8rzm79*_ga*Njk1NTI2Njg0LjE3MDQzMDI3NDA.*_ga_SYHREZS7XD*MTcwNjAxNTgzNy40LjEuMTcwNjAxNTg4NC4wLjAuMA..&layers%5B%5D=satellite&layers%5B%5D=geologyDetailed&layers%5B%5D=names)). The last time Hekla erupted was in 2000, but prior to that it had erupted approximately every 10 years. Notably, lava compositions from basalt to rhyolite have been erupted, but the origin of this compositional diversity is debated. One model proposes that the silicic melts are produced by melting of hydrothermally altered crust [(e.g., Sigmarsson et al., 2022)](https://doi.org/10.1007/s00410-021-01883-5), with compositionally intermediate melts formed by mixing between the mafic (mantle derived) basaltic melt and this silicic crustal melt. Alternatively, it is suggested that the entire compositional suite can be formed by fractional crystallisation of basaltic melt [(e.g., Geist et al., 2021)](http://www.doi.org/10.1093/petrology/egab001).

In this practical you will use thermodynamic modelling to assess whether fractional crystallisation is a viable model for explaining the diversity in magma compositions.

---

First, import the python packages required:

In [ ]:
import numpy as np
import pandas as pd
import magmaforge
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

`Magmaforge` is a python library for running thermodynamic calculations for modelling magmatic systems, and will make our task much easier.

## Part 1: Import data and plot magma compositions

A table of whole rock compositions from Hekla is contained in the csv file `hekla.csv`, which we can import:

In [ ]:
hekla_data = pd.read_csv('hekla.csv')
hekla_data['FeOT'] = hekla_data['FeOt']
hekla_data.head()

We can make some Harker plots to visualise the variation in the composition of the samples:

In [ ]:
fig, ax = plt.subplots()

ax.scatter(hekla_data['MgO'], hekla_data['CaO'])

ax.set_xlabel('MgO (wt%)')
ax.set_ylabel('CaO (wt%)')

plt.show()

### Q1.1: Using the code above as an example, make some more Harker plots with at least 3 other major element oxides

In [ ]:
# Your code here...

In [ ]:
# Your code here...

In [ ]:
# Your code here...

### Q1.2: Summarise the variability in the major elements. How might this be consistent with crystal fractionation or mixing with silicic crustal melts?

*Your answer here...*

## Part 2: Set up the thermodynamic calculation

Before we can begin to model the crystallisation of the magma we need to choose a composition to start with. This should be our best guess at what the most primitive melt in the system is. The sample with highest MgO will be a good starting point. In this context primitive means the magma that has undergone least crystallisation.

### Q2.1 Why is the sample with the highest MgO likely to be the most primitive?

*Your answer here...*

We can extract the row of the table that corresponds to the most primitive composition:

In [ ]:
most_primitive = hekla_data.loc[hekla_data.MgO.argmax()]
most_primitive

Now we can extract the oxide components and adjust the H$_2$O content to account for the lava having lost most of its H$_2$O to volcanic degassing:

In [ ]:
comp={
      'SiO2': most_primitive.SiO2,
      'TiO2': most_primitive.TiO2,
      'Al2O3': most_primitive.Al2O3,
      'Fe2O3': 0.0,
      'FeO':  most_primitive.FeOt,
      'MgO':  most_primitive.MgO,
      'CaO':  most_primitive.CaO,
      'Na2O': most_primitive.Na2O,
      'K2O':  most_primitive.K2O,
      'P2O5':  0.0,
      'H2O':  1.0, # You might want to try adjusting this later!
      }

We need to tell `magmaforge` what temperature to start the calculations at (in degrees Kelvin). A temperature of 1300˚C should be comfortably above the liquidus of the rock composition we have chosen:

In [ ]:
T0=1300 + 273.15

## Part 3: Do some calculations!

Now we need to set up a `magmaforge` system. We will start with a system at 6 kbar, which would correspond to a very deep magma chamber:

In [ ]:
sys = magmaforge.System(comp=comp, P=6000.0, T0=T0, model_name="v1.2",
                        O2_buffer='NNO', del_fO2=0)

Now we will run a crystallisation calculation. If the calculation runs for longer than 1 minute it is likely the algorithm has got stuck. Press stop and continue with the results calculated up to that point.

In [ ]:
sys.crystallize(method='equil', # You can change this to 'frac' to simulate fractional crystallisation
                fix_fO2=True, 
                Tstep=2)

We can visualise the model results:

In [ ]:
magmaforge.plot.magma_evolution(sys.history)

This isn't particularly helpful in identifying whether we have modelled the suite of data from Hekla. A better way to plot this up is to use a ternary plot:

In [ ]:
fig = plt.figure()
ax = magmaforge.plot.TernaryPlotAxes(fig)

# Plot the model result
tbl = sys.history.liquid_comp_table
tbl['FeOT'] = tbl['FeO'] + tbl['Fe2O3'] / (55.845 + 1.5*15.999) * (55.845 + 15.999)
ax.plot(magmaforge.plot.ternaryEndmembersFromCIPW(tbl, calcFeSpeciation=True)[0], lw=5, c='r')

# Plot the Hekla Data
ax.scatter(magmaforge.plot.ternaryEndmembersFromCIPW(hekla_data, calcFeSpeciation=True)[0], zorder=10)

Crystallisation at these conditions does not match the data well. Something to think about- why does the model path suddenly change direction?

Now try changing the pressure the crystallisation is happening at, and re-make the plots. What pressure works best?

How much silicic melt can be produced by this model (relative to the amount of basaltic magma you start with)?

Can you get a better fit by using fractional crystallisation? You can use the following to see how much liquid remains at the end of a fractional crystallisation calculation:

In [ ]:
sys.history.get_total_mass()